# Object detection model customization

## Setup

To install the libraries for customizing a model, run the following commands:

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Use the following code to import the required Python classes:

In [ ]:
from google.colab import files
import os
import json
from tqdm import tqdm
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

from google.colab import drive
import shutil

%matplotlib inline

### Colab Pro

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### Set paths

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
base_path = '/content/drive/MyDrive/'
source_path = base_path + 'Datasets/revisitop/rparis6k/data/'

dest_base_path = base_path + 'MyProject/rparis6k/'

train_dataset_path = dest_base_path + 'train/'
validation_dataset_path = dest_base_path + 'validation/'
test_dataset_path = dest_base_path + 'test/'

In [ ]:
#@title Create directories

os.makedirs(dest_base_path, exist_ok=True)

os.makedirs(train_dataset_path, exist_ok=True)
os.makedirs(validation_dataset_path, exist_ok=True)
os.makedirs(test_dataset_path, exist_ok=True)

os.makedirs(os.path.join(train_dataset_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(validation_dataset_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dataset_path, 'images'), exist_ok=True)

## Prepare data

### Copy images

In [ ]:
# FIXME:
def check_and_delete_augmented_images(folder_path):
    augmented_images = [f for f in os.listdir(folder_path) if 'aug' in f]

    if len(augmented_images) > 0:
        print(f"Found {len(augmented_images)} augmented images in {folder_path}.")
        user_input = input("Do you want to delete these images? (yes/no): ").strip().lower()

        if user_input == 'yes':
            # delete all the elements in the image folder
            # delete the labels.json file
            for img in tqdm(augmented_images, desc="Deleting augmented images"):
                img_path = os.path.join(folder_path, img)
                os.remove(img_path)
            print("Augmented images deleted successfully.")
        else:
            print("Deletion aborted by user.")
    else:
        print("No augmented images found.")

In [ ]:
# TODO: mention that the train set contain augmented images if it's the case
# TODO: use also a boolean value to memorize the aug data deletion

In [ ]:
# Function to copy images
def copy_images(file_list, dest_folder):
    with open(file_list, 'r') as f:
        lines = f.readlines()
        for line in tqdm(lines, desc=f"Copying images to {dest_folder}"):
            img_name = line.strip()
            src = os.path.join(source_path, img_name)
            dst = os.path.join(dest_folder, img_name)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)

# Copy images for each set
if os.listdir(train_dataset_path + 'images/') == [] and \
   os.listdir(validation_dataset_path + 'images/') == [] and \
   os.listdir(test_dataset_path + 'images/') == []:
    copy_images(dest_base_path + 'train.txt', train_dataset_path + 'images/')
    copy_images(dest_base_path + 'val.txt', validation_dataset_path + 'images/')
    copy_images(dest_base_path + 'test.txt', test_dataset_path + 'images/')
    print("Dataset division completed!\n")
else:
    print("One or more directories are not empty. Copy operation aborted.\n")

print(f"Number of images in train set: {len(os.listdir(train_dataset_path + 'images/'))}")
print(f"Number of images in validation set: {len(os.listdir(validation_dataset_path + 'images/'))}")
print(f"Number of images in test set: {len(os.listdir(test_dataset_path + 'images/'))}")

### Review data

Verify the dataset content by printing the categories from the `labels.json` file. There should be 13 total categories. Index 0 is always set to be the `background` class which may be unused in the dataset.

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

In [ ]:
#@title Visualize the training data [FIXME]
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

# TODO: it may be interesting if it shows two (or n) random images per category

def draw_outline(obj):
    obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])

def draw_box(ax, bb):
    patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
    draw_outline(patch)

def draw_text(ax, bb, txt, disp):
    text = ax.text(bb[0], (bb[1]-disp), txt, verticalalignment='top', color='white', fontsize=10, weight='bold')
    draw_outline(text)

def draw_bbox(ax, annotations_list, id_to_label, image_shape):
    for annotation in annotations_list:
        cat_id = annotation["category_id"]
        bbox = annotation["bbox"]
        draw_box(ax, bbox)
        draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)

def visualize(dataset_folder, max_examples=None):
    with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
        labels_json = json.load(f)

    images = labels_json["images"]
    cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
    image_annots = defaultdict(list)

    for annotation_obj in labels_json["annotations"]:
        image_id = annotation_obj["image_id"]
        image_annots[image_id].append(annotation_obj)

    if max_examples is None:
        max_examples = len(image_annots.items())

    n_rows = math.ceil(max_examples / 3)
    fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image

    for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
        ax = axs[ind//3, ind%3]
        img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
        ax.imshow(img)
        draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)

    plt.show()

visualize(train_dataset_path, 9)

### Create dataset

In [ ]:
# TODO: do I need this instruction ?

cache_dirs = ["/tmp/od_data/train", "/tmp/od_data/validation"]

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")

print(f"{'Training Dataset Size:':<25} {train_data.size:>4}")
print(f"{'Validation Dataset Size:':<25} {validation_data.size:>4}")

## Augmentation

### Augment data

In [ ]:
import albumentations as A
import numpy as np
import cv2

In [ ]:
# TODO: check the aug data deletion boolean value
# if true -> ask to execute augmentation
# if false -> ask to delete aug data before and then to execute augmentation (update train_data_path)

In [ ]:
def get_transform(set='train'):
    bboxes_params = A.BboxParams(format='coco', min_visibility=0.3, label_fields=['class_labels']) # TODO: check min_visibility

    if set == 'train':
        transform = A.Compose([ # TODO: update pipeline (?)
            # TODO: do I need to resize images?
            #A.RandomResizedCrop(height=640, width=640, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0), # TODO: check h,w
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.3),
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
            A.OneOf([
                A.MotionBlur(blur_limit=7, p=0.5),
                A.MedianBlur(blur_limit=7, p=0.5),
                A.GaussianBlur(blur_limit=7, p=0.5),
            ], p=0.3),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, border_mode=0, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # TODO: check
        ], bbox_params=bboxes_params)

    elif set == 'validation':
        transform = A.Compose([ # TODO: update pipeline
            # TODO: do I need to resize images?
            A.HorizontalFlip(p=0.5),
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # TODO: check
        ], bbox_params=bboxes_params)

    return transform

In [ ]:
with open(os.path.join(train_dataset_path, 'labels.json'), 'r') as f:
    train_json = json.load(f)

with open(os.path.join(validation_dataset_path, 'labels.json'), 'r') as f:
    val_json = json.load(f)

with open(os.path.join(test_dataset_path, 'labels.json'), 'r') as f:
    test_json = json.load(f)

n_images = max(train_json['images'][-1]['id'], val_json['images'][-1]['id'], test_json['images'][-1]['id'])
n_annotations = max(train_json['annotations'][-1]['id'], val_json['annotations'][-1]['id'], test_json['annotations'][-1]['id'])

In [ ]:
def clip_bbox(bbox, image_width, image_height):
    x_min, y_min, width, height = bbox

    x_min = max(0, min(x_min, image_width - 1)) # TODO: check -1
    y_min = max(0, min(y_min, image_height - 1)) # TODO: check -1
    width = min(width, image_width - x_min)
    height = min(height, image_height - y_min)

    return [x_min, y_min, width, height]

In [ ]:
def validate_bbox(bbox, image_width, image_height):
    x, y, w, h = bbox

    return 0 <= x < image_width and 0 <= y < image_height and x + w <= image_width and y + h <= image_height

In [ ]:
def apply_augmentation(image_path, bboxes, class_labels, output_path, output_filename, transform):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_height, image_width = image.shape[:2]

    # Apply the augmentation
    try:
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    except Exception as e:
        print(f"Error during transformation: {e}")
        return [], []

    # Save the augmented image
    augmented_image_path = os.path.join(output_path, output_filename)
    cv2.imwrite(augmented_image_path, cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

    return transformed['bboxes'], transformed['class_labels']

In [ ]:
def augment_dataset(input_path, output_path, transform, n_images, n_annotations, num_augmentations=5):
    # Load the original COCO JSON file
    with open(os.path.join(input_path, 'labels.json'), 'r') as f:
        coco_data = json.load(f)

    new_images = []
    new_annotations = []

    # Copy original images and annotations
    for img in tqdm(coco_data['images'], desc="Copying original images"):

        src_path = os.path.join(input_path, 'images', img['file_name'])
        dst_path = os.path.join(output_path, 'images', img['file_name'])
        shutil.copy2(src_path, dst_path)

        new_images.append(img)
        img_anns = [ann for ann in coco_data['annotations'] if ann['image_id'] == img['id']]
        new_annotations.extend(img_anns)

    '''debug'''
    print("Before augmentation:")
    print(f"Number of images: {len(new_images)}")
    print(f"Number of annotations: {len(new_annotations)}")

    # Apply augmentations
    for img in tqdm(coco_data['images'], desc="Augmenting images"):
        image_path = os.path.join(input_path, 'images', img['file_name'])

        annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img['id']]

        image = cv2.imread(image_path)
        image_height, image_width = image.shape[:2]

        for i in range(num_augmentations):
            bboxes = [ann['bbox'] for ann in annotations]
            class_labels = [ann['category_id'] for ann in annotations]

            # TODO: should I call the function clip_bbox() regardless of the function validate_bbox()?
            for bbox in bboxes:
                if not validate_bbox(bbox, image_width, image_height):
                    bboxes = [clip_bbox(bbox, image_width, image_height) for bbox in bboxes]

            new_filename = f"{os.path.splitext(img['file_name'])[0]}_aug_{i}.jpg"

            new_bboxes, new_class_labels = apply_augmentation(
                image_path, bboxes, class_labels,
                os.path.join(output_path, 'images'), new_filename, transform
            )

            new_img_id = n_images + 1
            new_images.append({
                'id': new_img_id,
                'file_name': new_filename
            })

            n_images = n_images + 1

            for bbox, cat_id in zip(new_bboxes, new_class_labels):
                new_annotations.append({
                    'id': n_annotations + 1,
                    'image_id': new_img_id,
                    'category_id': cat_id,
                    'bbox': bbox
                })

                n_annotations = n_annotations + 1

    '''debug'''
    print("After augmentation:")
    print(f"Number of images: {len(new_images)}")
    print(f"Number of annotations: {len(new_annotations)}")

    # Create the new COCO JSON file
    new_coco_data = {
        'categories': coco_data['categories'],
        'images': new_images,
        'annotations': new_annotations
    }

    # Save the new COCO JSON file
    with open(os.path.join(output_path, 'labels.json'), 'w') as f:
        json.dump(new_coco_data, f, indent=4)

    return n_images, n_annotations

In [ ]:
augmented_train_dataset_path = dest_base_path + 'train_augmented/'
augmented_validation_dataset_path = dest_base_path + 'validation_augmented/'

os.makedirs(os.path.join(augmented_train_dataset_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(augmented_validation_dataset_path, 'images'), exist_ok=True)

if os.listdir(augmented_train_dataset_path + 'images/') == []:
    n_images, n_annotations = augment_dataset(train_dataset_path, augmented_train_dataset_path, get_transform('train'), n_images, n_annotations, num_augmentations=5)
else:
    print("Augmentation on the training set has already been made.")

if os.listdir(augmented_validation_dataset_path + 'images/') == []:
    augment_dataset(validation_dataset_path, augmented_validation_dataset_path, get_transform('validation'), n_images, n_annotations, num_augmentations=5)
else:
    print("Augmentation on the validation set has already been made.")

In [ ]:
count1 = sum(1 for filename in os.listdir(os.path.join(augmented_train_dataset_path, 'images')) if any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg']))
count2 = sum(1 for filename in os.listdir(os.path.join(augmented_validation_dataset_path, 'images')) if any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg']))

print(f"Number of images in the train_augmented folder: {count1}") # TODO: make prettier
print(f"Number of images in the validation_augmented folder: {count2}") # TODO: make prettier

In [ ]:
#@title Visualize the augmented training data [FIXME]

visualize(augmented_train_dataset_path, 12)

### Rewrite train dataset

In [ ]:
# TODO: add if condition (if augmentation has been executed)

In [ ]:
shutil.rmtree("/tmp/od_data/augmented_train") # TODO: do I need this instruction ?
shutil.rmtree("/tmp/od_data/augmented_validation") # TODO: do I need this instruction ?

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(augmented_train_dataset_path, cache_dir="/tmp/od_data/augmented_train")
validation_data = object_detector.Dataset.from_coco_folder(augmented_validation_dataset_path, cache_dir="/tmp/od_data/augmented_validation")

print(f"{'New Training Dataset Size:':<25} {train_data.size:>6} images")
print(f"{'New Validation Dataset Size:':<25} {validation_data.size:>4} images")

## Retrain model

### Set retraining options

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG_I384

hparams = object_detector.HParams(
    learning_rate=0.01, # 0.015 (is it possible to implement a scheduler?)
    batch_size=64, # try 128, 256
    epochs=100,
    cosine_decay_epochs=100,
    cosine_decay_alpha=0.1,
    shuffle=True, # TODO: check
    #repeat=True, # TODO: check
    export_dir='exported_model'
)

model_options = object_detector.ModelOptions(
    l2_weight_decay=1e-4 # 3e-5
)

options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams,
    model_options=model_options
)

### Run retraining

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

### Evaluate the model performance

After training the model, evaluate it on validation dataset and print the loss and coco_metrics. The most important metric for evaluating the model performance is typically the "AP" coco metric for Average Precision.

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=32) # TODO: check batch_size
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=64) # TODO: check batch_size
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

## Export model

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label map.

In [ ]:
# TODO: do I need to remove the existing model first?

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

## Model quantization

Model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy.

This section of the guide explains how to apply quantization to your model. Model Maker supports two forms of quantization for object detector:
1. Quantization Aware Training: 8 bit integer precision for CPU usage
2. Post-Training Quantization: 16 bit floating point precision for GPU usage

### Quantization aware training (int8 quantization)
Quantization aware training (QAT) is a fine-tuning step which happens after fully training your model. This technique further tunes a model which emulates inference time quantization in order to account for the lower precision of 8 bit integer quantization. For on-device applications with a standard CPU, use Int8 precision. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) documentation.

To apply quantization aware training and export to an int8 model, create a `QATHParams` configuration and run the `quantization_aware_training` method. See the **Hyperparameters** section below on detailed usage of `QATHParams`.

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

The QAT step often requires multiple runs to tune the parameters of training. To avoid having to rerun model training using the `create` method, use the `restore_float_ckpt` method to restore the model state back to the fully trained float model(After running the `create` method) in order to run QAT again.

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Finally, us the `export_model` to export to an int8 quantized model. The `export_model` function will automatically export to either float32 or int8 model depending on whether `quantization_aware_training` was run.

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')

### Post-training quantization (fp16 quantization)

Post-training model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy. This approach reduces the size of the data processed by the model, for example by transforming 32-bit floating point numbers to 16-bit floats. Float16 quantization is reccomended for GPU usage. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) documentation.

First, import the MediaPipe Model Maker quantization module:

In [ ]:
from mediapipe_model_maker import quantization

Define a QuantizationConfig object using the `for_float16()` class method. This configuration modifies a trained model to use 16-bit floating point numbers instead of 32-bit floating point numbers. You can further customize the quantization process by setting additional parameters for the QuantizationConfig class.

In [ ]:
quantization_config = quantization.QuantizationConfig.for_float16()

Export the model using the additional quantization_config object to apply post-training quantization. Note that if you previously ran `quantization_aware_training`, you must first convert the model back to a float model by using `restore_float_ckpt`.

In [ ]:
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)
!ls -lh exported_model
files.download('exported_model/model_fp16.tflite')